In [2]:
import pandas as pd
import json
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Add src directory to path for k_base module
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from experiments.helpers import *
from analysis.plot_utils import set_paper_style, save_plots
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [11]:
def print_stats(method:str, df:pd.DataFrame):
    agent_names = df.agent_name.unique().tolist()
    
    for agent_name in agent_names:
        agent_df = df.loc[df["agent_name"]==agent_name]
        # agent_df = agent_df.replace(0.0, np.nan)
        print(f"Results for {agent_name} agent")
        print(f"    {agent_name} x personalization: {agent_df.relevance_personalization.mean()}")
        print(f"    {agent_name} x popularity: {agent_df.relevance_popularity.mean()}")
        print(f"    {agent_name} x sustainability: {agent_df.relevance_sustainability.mean()}")
        print(f"    {agent_name} x overall (all filters): {agent_df.overall_relevance.mean()}")

In [12]:
def print_stats_by_pop_levels(method, df):
    POP_LEVELS = ["all", "low", "medium", "high"]
    for pop_level in POP_LEVELS:
        if pop_level in ["low", "medium", "high"]:
            df_pop = df.loc[df["pop_level"]==pop_level]
        else:
            df_pop = df
        print(f"{'='*10} {method.upper()} RESULTS for {len(df_pop.query_id.unique())} queries (pop-level): {pop_level}")
        print_stats(method, df_pop)

In [13]:
def get_df(method="mami"):
    if method == "sasi":
        file_name = "../data/collab-rec-2026/analysis/gemini_sasi_scores_with_relevance.csv"
    else:
        file_name = f"../data/collab-rec-2026/analysis/gemini_{method}_aggressive_scores_with_relevance.csv"
    df = pd.read_csv(file_name)
    df[['reliability_score']] = df[['reliability_score']].fillna(value=1)
    df.rename(columns={"success_score": "agent_success_score"}, inplace=True)
    df["pop_level"] = df["query_id"].apply(get_pop_level)
    return df

In [14]:
mami_df = get_df("mami")
print_stats_by_pop_levels(method="mami", df=mami_df)

========== MAMI RESULTS for 326 queries (pop-level): all
Results for popularity agent
    popularity x personalization: 0.692418200408998
    popularity x popularity: 0.4161042944785276
    popularity x sustainability: 0.7406612133605999
    popularity x overall (all filters): 0.6209296182685753
Results for personalization agent
    personalization x personalization: 0.6974284253578732
    personalization x popularity: 0.40211656441717786
    personalization x sustainability: 0.7485173824130881
    personalization x overall (all filters): 0.6213369120654397
Results for sustainability agent
    sustainability x personalization: 0.6956288343558282
    sustainability x popularity: 0.40745398773006136
    sustainability x sustainability: 0.7376073619631903
    sustainability x overall (all filters): 0.6204626789366053
Results for moderator agent
    moderator x personalization: 0.6967791411042945
    moderator x popularity: 0.4089570552147239
    moderator x sustainability: 0.7434253578732

In [ ]:
mami_df["candidates"][0]

In [ ]:
mami_df_sus = mami_df.loc[mami_df['query_id'].str.contains('sus')]
print(len(mami_df_sus.query_id.unique()))
mami_df_sus["filters"][680]
mami_df_sus["candidates"][680]

In [ ]:
mami_df.relevance_sustainability.value_counts()

In [ ]:
mami_df_sus.head()

In [ ]:
print(mami_df_sus.relevance_sustainability.mean())
mami_df_sus.relevance_sustainability.value_counts()

In [15]:
masi_df = get_df("masi")
print_stats_by_pop_levels(method="masi", df=masi_df)

========== MASI RESULTS for 326 queries (pop-level): all
Results for popularity agent
    popularity x personalization: 0.6984151329243354
    popularity x popularity: 0.4092024539877301
    popularity x sustainability: 0.7540899795501022
    popularity x overall (all filters): 0.6223926380368099
Results for personalization agent
    personalization x personalization: 0.707157464212679
    personalization x popularity: 0.36595092024539877
    personalization x sustainability: 0.7840490797546013
    personalization x overall (all filters): 0.620756646216769
Results for sustainability agent
    sustainability x personalization: 0.6947852760736196
    sustainability x popularity: 0.38834355828220857
    sustainability x sustainability: 0.7498977505112475
    sustainability x overall (all filters): 0.6150051124744377
Results for moderator agent
    moderator x personalization: 0.7042433537832311
    moderator x popularity: 0.3806748466257669
    moderator x sustainability: 0.76952965235173

## SASI

In [16]:
sasi_df = get_df("sasi")
print_stats_by_pop_levels(method="sasi", df=sasi_df)

========== SASI RESULTS for 896 queries (pop-level): all
Results for sasi agent
    sasi x personalization: 0.7137090773809524
    sasi x popularity: 0.3592633928571428
    sasi x sustainability: 0.2652901785714286
    sasi x overall (all filters): 0.6219308035714286
========== SASI RESULTS for 299 queries (pop-level): low
Results for sasi agent
    sasi x personalization: 0.7113154960981048
    sasi x popularity: 0.06488294314381271
    sasi x sustainability: 0.23712374581939802
    sasi x overall (all filters): 0.5310479375696767
========== SASI RESULTS for 299 queries (pop-level): medium
Results for sasi agent
    sasi x personalization: 0.725752508361204
    sasi x popularity: 0.05083612040133779
    sasi x sustainability: 0.2682274247491639
    sasi x overall (all filters): 0.5445094760312151
========== SASI RESULTS for 298 queries (pop-level): high
Results for sasi agent
    sasi x personalization: 0.7040268456375839
    sasi x popularity: 0.9640939597315437
    sasi x sustainabi

## Top-Pop Baselines

In [ ]:
top_pop = pd.read_csv("../data/collab-rec-2026/analysis/top_pop_baseline_scores_with_relevance.csv")
top_pop["pop_level"] = top_pop["query_id"].apply(get_pop_level)
# top_pop = top_pop.loc[top_pop["query_id"].isin(mami_df.query_id.unique().tolist())]
print(top_pop.shape)
print_stats_by_pop_levels(method="topPop", df=top_pop)

## RandRec Baselines

In [ ]:
rand_rec = pd.read_csv("../data/collab-rec-2026/analysis/random_baseline_scores_with_relevance.csv")
# rand_rec = rand_rec.loc[rand_rec["query_id"].isin(df.query_id.unique().tolist())]
print(rand_rec.shape)
rand_rec.head()
print_stats_by_pop_levels(method="randRec", df=rand_rec)